In [7]:
from typing import Annotated

from typing_extensions import TypedDict

from langgraph.graph import StateGraph,START,END
from langgraph.graph.message import add_messages
from langchain.text_splitter import CharacterTextSplitter
from langchain_ollama import OllamaLLM, OllamaEmbeddings
from langchain_community.document_loaders import TextLoader
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import HumanMessage
from langchain.vectorstores import Chroma
from langchain.chains import ConversationalRetrievalChain
from config import settings
import os
from dotenv import load_dotenv
load_dotenv()

True

In [8]:
class State(TypedDict):
    # Messages have the type "list". The `add_messages` function
    # in the annotation defines how this state key should be updated
    # (in this case, it appends messages to the list, rather than overwriting them)
    messages:Annotated[list,add_messages]
llm = OllamaLLM(model=settings.OLLAMA_MODEL, base_url=settings.OLLAMA_BASE_URL)
embeddings = OllamaEmbeddings(model=settings.EMBEDDING_MODEL, base_url=settings.OLLAMA_BASE_URL)

In [9]:
# Split documents into chunks
loader = TextLoader(settings.DATA_FILE)
text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=100)
documents = loader.load()
docs = text_splitter.split_documents(documents)

# Setup Chroma as vectorstore
# persist_directory = "chroma_db"  # or any other persistent directory

persist_directory = os.path.abspath("chroma_db")

vectorstore = Chroma.from_documents(
    documents=docs,
    embedding=embeddings,
    collection_name= 'rag-chroma',
    persist_directory=persist_directory,
)

# (Optional) Persist to disk
vectorstore.persist()

# Create retriever
retriever = vectorstore.as_retriever()

In [22]:


custom_prompt =  ChatPromptTemplate.from_template("""
You are a precise accounting information assistant for GL.
Use only the provided financial documentation to answer questions factually.
If information isn't available, respond professionally with:
"I don't have that specific information in our records - please contact our accounting team for assistance."

Answer requirements:
1. Provide only accounting/finance facts from the context
2. Never add conversational fluff or invitations to ask more
3. Quote exact figures/dates when available
4. Reference relevant forms/regulations where applicable
5. Keep answers under 3 sentences unless technical details require more

For unrelated questions:
"That question falls outside our accounting scope - please contact client services."

Context: {context}
Question: {question}
Concise accounting answer:""")

def chatbot(state: State) -> State:
    last_user_msg = state["messages"][-1].content
    context_docs = state["messages"][:-1]  # earlier messages are docs
    context = "\n".join([doc.page_content for doc in context_docs])
    formatted_prompt = custom_prompt.invoke({"context": context, "question": last_user_msg})
    response = llm.invoke(formatted_prompt)
    return {"messages": state["messages"]}


In [ ]:
# Creates and configures the state graph for handling queries and generating answers.

workflow = StateGraph(State)
# workflow.add_node("retrieve", retriever)
workflow.add_node("generate", chatbot)
# workflow.add_edge(START, "retrieve")
# workflow.add_edge("retrieve", "generate")
workflow.add_edge(START, "generate")
workflow.add_edge("generate", END)
# workflow.compile()
graph=workflow.compile()
response=graph.invoke({"messages": "Hi"})


In [24]:
print(response)

{'messages': [HumanMessage(content='Hi', additional_kwargs={}, response_metadata={}, id='8c5432aa-d825-4feb-980a-0a594dd1fe7b')]}


In [2]:
## Visualize the graph
from IPython.display import Image,display

try:
    display(Image(graph.get_graph().draw_mermaid_png()))
except Exception:
    pass

In [86]:
# Build chain
# qa_chain = ConversationalRetrievalChain.from_llm(
#     llm=llm,
#     retriever=retriever,
#     # memory=memory,
#     combine_docs_chain_kwargs={"prompt": custom_prompt.partial(firm_name="GL")},
#     verbose=False
# )

# result = qa_chain.invoke({"messages": [HumanMessage(content="What is GL software")]})
# for content in result["messages"]:
#     print(content)
# print(result["messages"])